In [1]:
import os 
import test_module
from collections import namedtuple
import numpy as np
import retrieval_run
import settings
import cloud_dic
import utils
import test_module


In [2]:
#instrument

fwhm=-1
wavelength_range=[1,2.8]
ndata=3
Rfile='nonuniformR.dat'



#retrieval_params

##gas
chemeq=0
# gaslist = ['h2o','co','tio','vo','crh','feh','na','k']
gaslist = ['h2o','co','tio','vo','crh','feh','k','na']
gastype_list=['U','U','U','U','U','N','N','N']
ptype=77  

## clouds

do_clouds=1
npatches=2
# cloudname = ['powerlaw cloud deck', 'grey cloud slab', 'Mie scattering cloud deck--Fe']

cloudpath='/Users/fw23aao/Downloads/cloud_pickles/'


cloud_name=['clear','powerlaw', 'grey', 'tholin_WS15.mieff']
cloud_type=['None',"deck","slab","deck"]

cloudpacth_index=[[2],[1],[1],[1]]
particle_dis=['','', '', 'hansan']



# cloud_name=['powerlaw', 'grey', 'ZnS_WS15.mieff']
# cloud_type=["Deck","slab","deck"]

# cloudpacth_index=[[1],[1],[1]]
# particle_dis=['hansan', 'hansan', 'hansan']


# ModelConfig:
do_fudge=1
samplemode='mcmc'

# instrument_instance = utils_dev.Instrument(fwhm,wavelength_range,ndata,Rfile)
# re_params = utils_dev.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloudname,cloudpacth_index,particle_dis)
# model_config_instance = utils_dev.ModelConfig(samplemode,do_fudge)
# io_config_instance = utils_dev.IOConfig()


instrument_instance = utils.Instrument(fwhm,wavelength_range,ndata,Rfile)
re_params = utils.Retrieval_params(samplemode,chemeq,gaslist,gastype_list,fwhm,do_fudge,ptype,do_clouds,npatches,cloud_name,cloud_type,cloudpacth_index,particle_dis)
model_config_instance = utils.ModelConfig(samplemode,do_fudge,cloudpath=cloudpath)
io_config_instance = utils.IOConfig()





model_config_instance.dist= 5.84
model_config_instance.xlist ='data/gaslistRox.dat'
model_config_instance.do_bff=0
model_config_instance.malk=1
model_config_instance.ch4=0
model_config_instance.update_dictionary()


In [3]:
obspec = np.asfortranarray(np.loadtxt("./examples/example_data/G570D_2MHcalib.dat",dtype='d',unpack='true')) # G570D_2MassJcalib.dat
args_instance = utils.ArgsGen(re_params,model_config_instance,instrument_instance,obspec)
settings.init(args_instance)

In [4]:
# args_instance.gasnames

In [5]:
args_instance.cloudflag

array([['powerlaw cloud deck', 'grey cloud slab',
        'Mie scattering cloud deck--tholin_WS15.mieff'],
       [0, 0, 0]], dtype=object)

In [6]:
np.shape(args_instance.cloudata),np.shape(args_instance.miewave),np.shape(args_instance.mierad)

((2, 3, 3, 196, 60), (196,), (60,))

In [7]:
args_instance.cloudata[0,2,0],np.shape(args_instance.cloudata[0,2,0])

(array([[0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.21814097e+00, 1.21324181e+00, 1.20872031e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.20300175e+00, 1.19883326e+00, 1.19503705e+00],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         1.18848917e+00, 1.18483886e+00, 1.18153888e+00],
        ...,
        [6.67740000e-08, 1.49464000e-07, 4.27693000e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [9.46880000e-08, 2.11946000e-07, 6.06497000e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [1.27212000e-07, 2.84749000e-07, 8.14843000e-07, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00]]),
 (196, 60))

In [8]:
args_instance=settings.runargs
all_params,all_params_values =utils.get_all_parametres(re_params.dictionary)

if re_params.samplemode=='mcmc':
    
    model_config_instance.ndim=len(all_params)
    model_config_instance.nwalkers=len(all_params)*16

    r2d2 = (71492e3)**2. / (model_config_instance.dist * 3.086e+16)**2.
    re_params.dictionary['refinement_params']['params']['r2d2']['distribution']=['normal',r2d2,0.1*r2d2]


    def tolerance_parameter_customized_distribution(x):
        return np.log10((np.random.rand(x)* (max(args_instance.obspec[2,:]**2)*(0.1 - 0.01))) + (0.01*min(args_instance.obspec[2,10::3]**2))) 

    for i in range(len(all_params)):
        if all_params[i].startswith('tolerance_parameter'):
            re_params.dictionary['refinement_params']['params'][all_params[i]]['distribution']=['customized',tolerance_parameter_customized_distribution]

    if model_config_instance.fresh == 0:
        p0=utils.MC_P0_gen(re_params.dictionary,model_config_instance,args_instance)

In [9]:
p0

array([[-5.05715246e+00, -3.46433278e+00, -4.01640737e+00, ...,
         3.51973886e-02, -1.31676455e+00,  1.53648786e-01],
       [-3.56483635e+00, -4.01521298e+00, -5.03232549e+00, ...,
         3.35105088e-03, -1.43115411e+00,  1.31450405e-01],
       [-3.59704536e+00, -3.46621183e+00, -3.90818396e+00, ...,
         6.72138918e-02, -1.42866104e+00,  3.11438667e-01],
       ...,
       [-3.75769824e+00, -2.98583467e+00, -4.17630393e+00, ...,
         1.41741649e-01, -1.43052201e+00,  2.06855101e-01],
       [-4.36400763e+00, -3.69816713e+00, -4.23191957e+00, ...,
         1.83703299e-01, -1.46568740e+00,  2.32551843e-01],
       [-3.61202532e+00, -4.30093457e+00, -3.91172790e+00, ...,
         2.09210812e-01, -1.37895238e+00,  1.39622980e-01]])

In [10]:
print(args_instance.cloudflag.flags['F_CONTIGUOUS'])
print(args_instance.cloudata.flags['F_CONTIGUOUS']) 
print(args_instance.miewave.flags['F_CONTIGUOUS']) 
print(args_instance.mierad.flags['F_CONTIGUOUS'])


True
True
True
True


In [ ]:
gnostics = 0

shiftspec, photspec, tauspec,cfunc = test_module.modelspec(p0[0,:],re_params,args_instance,gnostics)


In [ ]:
outspec,tmpclphotspec,tmpophotspec,cf = forwardmodel.marv(temp,logg,R2D2,gasnames,gasmass,logVMR,pcover,
do_clouds,cloudflag,cloudata,miewave,mierad,cloudrad,cloudsig,cloudprof,inlinetemps,press,inwavenum,linelist,\
cia,ciatemps,use_disort,clphot,ophot,make_cf,do_bff,bff)

    

In [ ]:
forwardmodel.marv(temp,logg,R2D2,gasnum,logVMR,pcover,do_clouds,cloudnum,cloudrad,cloudsig,cloudprof,
                  inlinetemps,press,inwavenum,linelist,cia,ciatemps,use_disort,clphot,ophot,make_cf,do_bff,bff)

In [ ]:
subroutine marv(temp,logg,R2D2,ingasname,molmass,logVMR,pcover,&
     do_clouds,incloudname,clouddata,miewave,mierad, &
     cloudrad,cloudsig,cloudprof,&
     inlinetemps,inpress,inwavenum,inlinelist,cia,ciatemps,&
     use_disort,make_cl_pspec,make_oth_pspec,make_cf,do_bff,bff,outspec,&
     cl_phot_press,oth_phot_press,cfunc)